# Neural Networks - Homework

__Métodos Intensivos de Computación Estadística__

Juan Sebastián Corredor Rodriguez - jucorredorr@unal.edu.co

See my [Github Account](https://github.com/juanse1608) to know more about me and my projects.

## Introduction

Los datos del archivo de Binary Matrix abstracts.csv contiene la matrix document-término del conjunto de datos de abstracts de la tarea 1. Asuma la dimensión $m = 10$ y haga un anáalisis de compontes principales y estime los vectores $\theta_i, i = 1, \dots N$ dados por las primeras m componentes principales. Use las herramientas de diferenciación automática y numérica disponibles en su lenguaje de programación de preferencia. Incluya todos sus códigos por separado.